In [1]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

from openinference.instrumentation import using_attributes
from openinference.instrumentation.langchain import LangChainInstrumentor

from langchain_ollama import ChatOllama

In [2]:
endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
tracer_provider.add_span_processor(SimpleSpanProcessor(ConsoleSpanExporter()))

LangChainInstrumentor().instrument(tracer_provider=tracer_provider)


In [ ]:
import os
import phoenix as px

os.environ["PHOENIX_TRACING"] = "true"
px.launch_app()

In [3]:


if __name__ == "__main__":
    with using_attributes(
        session_id="my-test-session",
        user_id="my-test-user",
        metadata={
            "test-int": 1,
            "test-str": "string",
            "test-list": [1, 2, 3],
            "test-dict": {
                "key-1": "val-1",
                "key-2": "val-2",
            },
        },
        prompt_template="Who won the soccer match in {city} on {date}",
        prompt_template_version="v1.0",
        prompt_template_variables={
            "city": "Johannesburg",
            "date": "July 11th",
        },
        tags=["tag-1", "tag-2"],
    ):
        prompt_template = "Tell me a {adjective} joke"
        prompt = PromptTemplate(input_variables=["adjective"], template=prompt_template)

        llmmodel = ChatOllama(model="llama3.2:latest", temperature=0)
        llm = LLMChain(llm=llmmodel, prompt=prompt, metadata={"category": "jokes"})
        completion = llm.predict(adjective="funny", metadata={"variant": "funny"})
        print(completion)

C:\Users\Muthu\AppData\Local\Temp\ipykernel_1956\1127603484.py:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm = LLMChain(llm=llmmodel, prompt=prompt, metadata={"category": "jokes"})


{
    "name": "ChatOllama",
    "context": {
        "trace_id": "0x57d98f08c84310cd8d7e2dc2cbd3cbae",
        "span_id": "0xdd54f7a9e8e93857",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x8041c0a5849822f3",
    "start_time": "2025-02-06T07:02:58.855861Z",
    "end_time": "2025-02-06T07:03:00.665872Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "session.id": "my-test-session",
        "user.id": "my-test-user",
        "tag.tags": [
            "tag-1",
            "tag-2"
        ],
        "llm.prompt_template.template": "Who won the soccer match in {city} on {date}",
        "llm.prompt_template.version": "v1.0",
        "llm.prompt_template.variables": "{\"city\": \"Johannesburg\", \"date\": \"July 11th\"}",
        "input.value": "{\"messages\": [[{\"lc\": 1, \"type\": \"constructor\", \"id\": [\"langchain\", \"schema\", \"messages\", \"HumanMessage\"], \"kwargs\": {\"content\": \"Tell me a funny joke\"